# Introduçãos as CNNs no Keras e ilustraçãos resultados obtidos para cada camada da DNN

Como funções de ativação, serão utilizados a função ReLU e a Sigmóide. 

**Sigmóide:**
A função sigmoide tem a forma de um "S". Ela é definida pela seguinte equação:

$$S(x) = \frac{1}{1+e^{-x}}$$

![](https://sebastianraschka.com/images/faq/logisticregr-neuralnet/sigmoid.png)

**ReLU:**
A função ReLU também chamda de fnção rampa é definida pela seguinte equação:

$$f(x) = max(0,x)$$

Simplesmente, ela é ativada caso o valor de entrada é maior que zero não alterando o mesmo, caso contrário o valor de entrada é multiplicado por zero na saída.

![](http://csci431.artifice.cc/images/relu.png)

# Importação dos Módulos

In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Input

Using Theano backend.
Using gpu device 0: Tesla K40c (CNMeM is disabled, cuDNN Version is too old. Update to v5, was 2000.)


# Definição da CNN


In [2]:
def get_cnn_model():
    
    # Tamanho da entrada
    img_rows = 5
    img_cols = 5
    
    # Pesos para as camadas de convolução e densa
    W = np.ones(shape=(2,2)).reshape(1,1,2,2)
    bias = np.ones(1)*.1
    W_dense = np.ones(shape=(4)).reshape(4,1)
    
    inputs = Input((1, img_rows, img_cols), name = 'input')
    conv1 = Convolution2D(1, 2, 2, weights = [W,bias],
                          name = 'conv1', border_mode='valid')(inputs) # default: subsample=(1, 1)  
    actv1 = Activation('relu', name = 'actv1')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), name = 'pool1')(actv1) # default: pool_size=(2, 2) 
    flat = Flatten(name = 'flat')(pool1)
    dense1 = Dense(1, weights = [W_dense,bias], name = 'dense1')(flat)
    out = Activation('sigmoid', name = 'out')(dense1)

    model = Model(input=inputs, output=out)
    
    return model

# Resultados da CNN para uma dada entrada

In [3]:
# Leitura do modelo
model = get_cnn_model()

# Entrada
X = np.array([[[[1,0,0,0,0],
                [0,1,0,0,0],
                [0,0,-1,0,0],
                [0,0,0,-1,0],
                [0,0,0,0,1]]]])



print('-'*30)
print "Número de camadas:", len(model.layers)
print('-'*30)

# Resultados para cada camada
i = 1
for layer in model.layers:
    intermediate_layer_model = Model(input=model.input,output=layer.output)
    intermediate_output = intermediate_layer_model.predict(X)
    print('-'*80)
    print "Saída da camada", i, ":", layer.name
    print('-'*80)
    print intermediate_output
    i+=1

------------------------------
Número de camadas: 7
------------------------------
--------------------------------------------------------------------------------
Saída da camada 1 : input
--------------------------------------------------------------------------------
[[[[ 1.  0.  0.  0.  0.]
   [ 0.  1.  0.  0.  0.]
   [ 0.  0. -1.  0.  0.]
   [ 0.  0.  0. -1.  0.]
   [ 0.  0.  0.  0.  1.]]]]
--------------------------------------------------------------------------------
Saída da camada 2 : conv1
--------------------------------------------------------------------------------
[[[[ 2.0999999   1.10000002  0.1         0.1       ]
   [ 1.10000002  0.1        -0.89999998  0.1       ]
   [ 0.1        -0.89999998 -1.89999998 -0.89999998]
   [ 0.1         0.1        -0.89999998  0.1       ]]]]
--------------------------------------------------------------------------------
Saída da camada 3 : actv1
--------------------------------------------------------------------------------
[[[[ 2.099

In [4]:
print model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input (InputLayer)               (None, 1, 5, 5)       0                                            
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 1, 4, 4)       5           input[0][0]                      
____________________________________________________________________________________________________
actv1 (Activation)               (None, 1, 4, 4)       0           conv1[0][0]                      
____________________________________________________________________________________________________
pool1 (MaxPooling2D)             (None, 1, 2, 2)       0           actv1[0][0]                      
___________________________________________________________________________________________